# Analysis of the language parameter



In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
sns.set_style("darkgrid")
from tqdm.notebook import tqdm, trange
from pandas.plotting import scatter_matrix

In [10]:
path = '../data/' 
hotels = pd.read_csv(path + '/features_hotels.csv')
data = pd.read_csv(path + 'data_language_utility.csv')

In [11]:
data = data.drop(['avatar_id','request_number'],axis=1)

In [12]:
data

,city,date,language,mobile,hotel_id,price,stock
0,valletta,1,maltese,1,334,79,0
1,valletta,1,maltese,1,400,87,8
2,valletta,1,maltese,1,732,176,1
3,valletta,1,maltese,1,988,160,0
4,valletta,1,maltese,1,521,84,0
...,...,...,...,...,...,...,...
87119,amsterdam,21,spanish,0,460,84,0
87120,amsterdam,21,spanish,0,971,160,22
87121,amsterdam,21,spanish,0,442,163,38
87122,amsterdam,21,spanish,0,517,117,28
